In [1]:
import torch 
from torch import nn
from d2l import torch as d2l

In [4]:
# Cross-correlation operation

def crossCorr2d(x, k):
    h, w = k.shape
    y = torch.zeros((x.shape[0] - h + 1, x.shape[1] - w + 1))
    for i in range(y.shape[0]):
        for j in range(y.shape[1]):
            y[i, j] = (x[i:i+h, j:j+w] * k).sum()
    return y

In [5]:
## Convolution layers

class Conv2d(nn.Module):
    def __init__(self, k_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(k_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return crossCorr2d(x, self.weight) + self.bias

In [15]:
conv2d = nn.LazyConv2d(1, kernel_size=(1, 2), bias=False) ## 2d conv layer with 1 output channel and kernel of (1, 2)

x = torch.ones((6, 8))
x[:, 2:6] = 0
x
k = torch.tensor([[1.0, -1.0]])
y  = crossCorr2d(x, k)

x = x.reshape((1, 1, 6, 8)) #(batch, channel, height, width)
y = y.reshape((1, 1, 6, 7))
lr = 3e-2

for i in range(10):
    y_hat = conv2d(x)
    l = (y_hat - y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i + 1}, loss {l.sum():.3f}')

epoch 2, loss 20.224
epoch 4, loss 7.227
epoch 6, loss 2.783
epoch 8, loss 1.110
epoch 10, loss 0.450


In [16]:
## Padding
def comp_conv2d(conv2d, x):
    x = x.reshape((1, 1) + x.shape) # batch and no of channels are 1
    y = conv2d(x)
    return y.reshape(y.shape[2:])   # strip the first two dimensions: batch and channel

conv2d = nn.LazyConv2d(1, kernel_size=3, padding=1) # 1row and 1 column padded on either sides
x = torch.rand(size=(8,8))
comp_conv2d(conv2d, x).shape

/home/drishya/miniconda3/envs/d2l/lib/python3.9/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([8, 8])

In [17]:
conv2d = nn.LazyConv2d(1, kernel_size=3, padding=1, stride=2) # set the strides to both height and width by 2; halving the i/p h and w
comp_conv2d(conv2d, x).shape

/home/drishya/miniconda3/envs/d2l/lib/python3.9/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([4, 4])

In [18]:
## Multi channel input and output

def corr2d_multi_in(X, K):  ## Multiple input channels
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [19]:
## multi channel output
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [20]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [21]:
## Implementing 1 x 1 convolution layer

def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h*w))
    K = K.reshape((c_o, c_o))
    y = torch.matmul(K, X)
    return y.reshape((c_o, h, w))

In [24]:
## Maximum Pooling

def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(y.shape[0]):
        for j in range(y.shape[1]):
            if mode == 'max':
                y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                y[i, j] = X[i: i + p_h, j: j + p_w].max()
    return y

X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2)) # Since nothing mentioned it gives max pooling

tensor([[4., 5.],
        [7., 8.]])

In [25]:
pool2d(X, (2, 2), 'avg')

tensor([[4., 5.],
        [7., 8.]])